run the previous code

In [1]:
import pandas as pd

SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTC-USD"


def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

main_df = pd.DataFrame() # begin empty

#load the 4 coins data
ratios = ["BTC-USD", "LTC-USD", "BCH-USD", "ETH-USD"]  # 4 coins
for ratio in ratios:  # begin iteration
    print(ratio)
    dataset = f'D:/Rama/AI Coaching/crypto_data/{ratio}.csv'  
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])  

    # rename volume and close to include the ticker so we can still which close/volume is which:
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)

    df.set_index("time", inplace=True)  # set time as index so we can join them on this shared time
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]  # focus only on price and volume

    if len(main_df)==0:  # if the dataframe is empty
        main_df = df  # then it's just the current df
    else:  # otherwise, join this data to the main one
        main_df = main_df.join(df)

#clean the data
main_df.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
main_df.dropna(inplace=True)
print(main_df.head())  

main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))


print(main_df.head()) 

BTC-USD
LTC-USD
BCH-USD
ETH-USD
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
1528968960    6480.000000        1.490900      96.519997       16.991997   

            BCH-USD_close  BCH-USD_volume  ETH-USD_close  ETH-USD_volume  
time                                                                      
1528968720     870.859985       26.856577      486.01001       26.019083  
1528968780     870.099976        1.124300      486.00000        8.449400  
1528968840     870.789978        1.749862      485.75000       26.994646  
1528968900     870.000000        1.680500      486.00000    

seperate out the validation of sample data by taking the last 5% of the data.

In [2]:
times = sorted(main_df.index.values)#values:convert to numpy array
last_5pct = sorted(main_df.index.values)[-int(0.05*len(times))]#5% of times

validation_main_df = main_df[(main_df.index >= last_5pct)] #validation data is the index in the last 5%
main_df = main_df[(main_df.index < last_5pct)] #make the main_df into all the data up to last 5%

make a function to balance and normalize the data.

In [3]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn import preprocessing
from collections import deque
import numpy as np
import random

def preprocess_df(df):
    #remove the future column, because the target is called 'target'
    df= df.drop('future', 1)
    
    for col in df.columns:
        #normalize all the data except the target
        if col != 'target':
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col]= preprocessing.scale(df[col].values)#scale between 0 and 1
    
    df.dropna(inplace=True)
    
    #create actual sequence
    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences.
                                       # deque keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # the sequence must be 60 sequences
            sequential_data.append([np.array(prev_days), i[-1]]) 

    random.shuffle(sequential_data)  # shuffle for good measure.
